In [5]:
import locale

def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

!pip install git+https://github.com/openai/whisper.git
!pip install jiwer
!pip install torchaudio

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-m7ua34m_
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-m7ua34m_
  Resolved https://github.com/openai/whisper.git to commit 173ff7dd1d9fb1c4fddea0d41d704cfefeb8908c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [12]:
import whisper
import torchaudio
import torch

In [13]:
def transcribe_audio(audio_file_path, model_name="base.en", device="cuda" if torch.cuda.is_available() else "cpu"):
    """
    Transcribes an audio file using the Whisper model.

    Args:
        audio_file_path (str): Path to the audio file.
        model_name (str, optional): Name of the Whisper model to use. Defaults to "base.en".
        device (str, optional): Device to run the model on ("cuda" or "cpu"). Defaults to "cuda" if available, else "cpu".

    Returns:
        str: The transcribed text.
    """

    # Load the audio file
    audio, sample_rate = torchaudio.load(audio_file_path)

    # Check if audio has more than one channel and convert to mono if needed
    if audio.shape[0] > 1:
        audio = torch.mean(audio, dim=0, keepdim=True)

    # Pad/trim and move to device
    audio = whisper.pad_or_trim(audio.flatten()).to(device)
    mel = whisper.log_mel_spectrogram(audio)

    # Load the Whisper model
    model = whisper.load_model(model_name, device=device)

    # Set decoding options
    options = whisper.DecodingOptions(language="en", without_timestamps=True)

    # Decode the audio
    result = model.decode(mel, options)

    # Access the transcribed text using the 'text' attribute
    return result.text # Changed from result["text"] to result.text

In [14]:
# Block 4: Load and Process Audio
import torch

audio_file_path = "WTFB1.mp4"  # Set your audio file path here
audio, sample_rate = torchaudio.load(audio_file_path)

# Check if audio has more than one channel and convert to mono if needed
if audio.shape[0] > 1:
    audio = torch.mean(audio, dim=0, keepdim=True)

# Define the device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") # Define DEVICE

audio = whisper.pad_or_trim(audio.flatten()).to(DEVICE)
mel = whisper.log_mel_spectrogram(audio)

In [15]:
transcript = transcribe_audio(audio_file_path, device=DEVICE)
print(transcript)

# Save the transcript to a file
with open("output.txt", "w") as f:
    f.write(transcript)

You're a... ...an essential person that has power. And that's a less than than than this. Yes, you're so bad.


In [16]:
# Block 6: Print Result and Save to File
# --- Get the transcription ---
transcript = transcribe_audio(audio_file_path, device=DEVICE) # Call the function to get the result

# --- Print and save the result ---
print(transcript) # Use the returned transcript

import os

# Get audio file name and construct output file path
audio_file_name, audio_file_ext = os.path.splitext(audio_file_path)  # Use the same variable
output_file_path = audio_file_name + "_transcribed.txt"  # Add '_transcribed' to filename

# Save the transcription to the file
with open(output_file_path, "w") as f:
    f.write(transcript)  # Save the transcript

print(f"Transcription saved to: {output_file_path}")

You're a... ...an essential person that has power. And that's a less than than than this. Yes, you're so bad.
Transcription saved to: WTFB1_transcribed.txt
